In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd 
import numpy as np


# Bu şekilde yapmamın sebebi verileri çekecegim sitenin beni bot olarak algılamasının önüne geçmesini sağlamak
# User-Agent başlığı, web tarayıcısı veya HTTP isteğini gönderen uygulamanın tarayıcı veya istemci hakkında bilgi içerir
# HTTP isteği gönderirken, gerçek bir tarayıcı gibi görünmeyi amaçlar. 
headers_param = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
}
url = "https://www.vatanbilgisayar.com/bilgisayar/"

# Web sayfasını çek
response = requests.get(url, headers=headers_param)

# Print the status code
print("Status Code:", response.status_code)


Status Code: 200


In [2]:
url_list = []
prices_list = []
computer_specs_titles = []
computer_specs_values = []

# Birden çok bilgisayarın verisini çekeceğimiz için sayfaları gezmeliyiz. Bu sayfalarda gezinerek tüm URL'leri çekeceğiz
for i in range(1):  # Rangede default step 1 olarak kabul ediliyor.
    url = "https://www.vatanbilgisayar.com/bilgisayar/?page=" + str(i) # Burada her bir sayfayı gezmesini saglıyoruz
    temp = requests.get(url)  # Verdigimiz Url 'ine GET isteginde bulunuyoruz.
    source = BeautifulSoup(temp.content, "lxml") #BeautifulSoup kütüphanesini kullanarak web sayfasının HTML içeriğini analiz eder.
# temp.content ifadesi, HTTP yanıtının içeriğini temsil eder. "lxml" ise kullanılan HTML analiz motorudur.
# Bu motor, HTML belgesini tree yapısına çevirerek içerisinde gezinmeyi ve belirli öğeleri bulmayı sağlar. 


    computer_urls = source.find_all("a", attrs={"class": "product-list__link"})
    for computer_url in computer_urls:
        full_computer_url = "https://www.vatanbilgisayar.com/" + computer_url.get("href") # Teknik özelliklere gitmesini sağladım
        url_list.append(full_computer_url) # Listenin sonuna her bir url'i ekledim

        temp_computer = requests.get(full_computer_url)
        source_computer = BeautifulSoup(temp_computer.content, "lxml")

        tech_specs_link = source_computer.find("a", text="Teknik Özellikler")

        if tech_specs_link:
            tech_specs_url = full_computer_url + tech_specs_link.get("href")
            print(tech_specs_url)
            temp_tech_specs_url = requests.get(tech_specs_url)
            source_tech_specs = BeautifulSoup(temp_tech_specs_url.content, "lxml")

            # H3 başlığı "İşlemci Özellikleri" olanları seç
            h3_titles = source_tech_specs.find_all("h3", class_="title title--feature")
            for h3_title in h3_titles:
                if h3_title.text.strip() == "İşlemci Özellikleri":
                    # product-table'i seç
                    properties = h3_title.find_next("table", class_="product-table")
                    rows = properties.find_all("tr")

                    for row in rows:
                        td_elements = row.find_all("td")

                        if len(td_elements) >= 2:
                            prop_title = td_elements[0].text.strip()
                            prop_value = td_elements[1].find("p").text.strip()

                            if "İzle" in prop_value:
                                continue

                            computer_specs_titles.append(prop_title)
                            computer_specs_values.append(prop_value)

    computer_prices = source.find_all("span", attrs={"class": "product-list__price"})
    # 10-34 yazılmasının sebebi ilk 10 veri mock data gibi görünüyor. Hocaya sorabilirsiniz.
    # Her URL'e doğru fiyatın gelmesi için ilk 10 veri alınmadı. 34 ise 34'ten sonraki URL'lere ait olmayan veriler.
    start_index = 10
    end_index = 34
    selected_prices = computer_prices[start_index:end_index]
    for price in selected_prices:
        prices_list.append(price.text)


C:\Users\Skyby\AppData\Local\Temp\ipykernel_17344\523659043.py:23: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tech_specs_link = source_computer.find("a", text="Teknik Özellikler")


https://www.vatanbilgisayar.com//honor-pad-x8-2ghz-4gb-64gb-10-1-android-tablet.html#urun-ozellikleri
https://www.vatanbilgisayar.com//asus-vivobook-15-12-nesil-core-i5-1235u-8gb-512gb-ssd-15-6inc-w11-140352.html#urun-ozellikleri
https://www.vatanbilgisayar.com//lenovo-tab-m9-1-8ghz-4gb-64gb-9inch-hd-android-tablet.html#urun-ozellikleri
https://www.vatanbilgisayar.com//lenovo-ideapad-slim-3-12-nesil-core-i5-12450h-8gb-512gb-ssd-15-6inc-w11.html#urun-ozellikleri
https://www.vatanbilgisayar.com//honor-pad-x9-2-8ghz-4gb-128gb-11-5-android-tablet.html#urun-ozellikleri
https://www.vatanbilgisayar.com//samsung-galaxy-tab-a9-64gb-4gb-11-android-tablet-sm-x210nzaatur-graphite.html#urun-ozellikleri
https://www.vatanbilgisayar.com//9-nesil-yeni-ipad-64gb-wifi-spacegray-10-2-retina-bluetooth-10saatekadarpilomru.html#urun-ozellikleri
https://www.vatanbilgisayar.com//lenovo-ideapad-slim-3-12-nesil-core-i5-12450h-8gb-256gb-ssd-15-6inc-w11.html#urun-ozellikleri
https://www.vatanbilgisayar.com//asus-v

In [5]:
df_urls=pd.DataFrame()
df_urls["urls"]=url_list
df_urls["prices"]=prices_list

In [6]:
df_urls.head()

,urls,prices
0,https://www.vatanbilgisayar.com//honor-pad-x8-...,3.195
1,https://www.vatanbilgisayar.com//asus-vivobook...,16.037
2,https://www.vatanbilgisayar.com//lenovo-tab-m9...,4.000
3,https://www.vatanbilgisayar.com//lenovo-ideapa...,19.701
4,https://www.vatanbilgisayar.com//honor-pad-x9-...,6.368


In [7]:
# Bulunan veri sayısı
computer_count = len(url_list)

In [8]:
# Bulunan özellik başlıklarının benzersizlerini bulma
columns=np.array(computer_specs_titles) # prpTitles ımı  np daki array metodu ile arraye ceviriyoruz
columns=np.unique(columns) # bunların unique olanlarını bulacagız.Böylelikle titlelardan sadece birer tane alacagız
new_feature = "Marka"
columns = np.append(columns, new_feature)
print(columns)

['Marka']


In [9]:
# Titleları kullanarak uri ve fiyat ile birlikte yeni bir data frame oluşturacagız
# Burada columns=columns demek ben dataframe oluşturacagım featureları yani column ları yukarıdan gelen column larımı buna ata diyoruz
df=pd.DataFrame(columns=columns) 
# Burada ise url ve price diye  columns ekliyoruz.Bunlar da daha öncesinde çekmiş oldugumuz veri listesi
df["url"]=url_list
df["price"]=prices_list


In [10]:
# Oluşturulan data frame gösterme
df.head()

,Marka,url,price
0,NaN,https://www.vatanbilgisayar.com//honor-pad-x8-...,3.195
1,NaN,https://www.vatanbilgisayar.com//asus-vivobook...,16.037
2,NaN,https://www.vatanbilgisayar.com//lenovo-tab-m9...,4.000
3,NaN,https://www.vatanbilgisayar.com//lenovo-ideapa...,19.701
4,NaN,https://www.vatanbilgisayar.com//honor-pad-x9-...,6.368


In [11]:
for i in range(0, computer_count):  # Url sayısı kadar dönecek
    url = df['url'].loc[i] # DataFrame içinde 'url' adlı bir sütunu seçtik ardından
    #  DataFrame içindeki belirli bir indeksi seçtik i indeksiyle ilgili olan 'url' sütunundaki değeri alır.
    temp = requests.get(url)  # Aldıgımız url GET isteginde bulunur
    source = BeautifulSoup(temp.content, "lxml")
    # GET isteginde bulundugumuz url'in içerigini  BeautifulSoup ile analiz ettik ve içerigini aldık.
    
    
    h1_title = source.find("h1", class_="product-list__product-name")
    
    if h1_title:
        # Eğer H1 başlığı bulunursa, ürünün isim değerini al dedik
        product_name = h1_title.text.strip()
        first_word = product_name.split()[0]

        # Marka değerini DataFrame'e ekle.Çünkü sitede sadece Marka'nın oldugu bir durum yok
        df.loc[i, "Marka"] = first_word
    
    # H3 başlığı "İşlemci Özellikleri" olanları seçtiriyorum
    h3_titles = source.find_all("h3", class_="title title--feature")
    for h3_title in h3_titles:
        if h3_title.text.strip() == "İşlemci Özellikleri":
            # product-table'i seç
            properties = h3_title.find_next("table", class_="product-table")
            rows = properties.find_all("tr")

            for row in rows:
                td_elements = row.find_all("td")

                if len(td_elements) >= 2:
                    prop_title = td_elements[0].text.strip()
                    prop_value = td_elements[1].find("p").text.strip()

                    if "İzle" in prop_value:
                        continue
                    df.loc[i, prop_title] = prop_value  # Assuming prop is defined somewhere
                    computer_specs_titles.append(prop_title)
                    computer_specs_values.append(prop_value)

In [13]:
df.head(20)

,Marka,url,price
0,Honor,https://www.vatanbilgisayar.com//honor-pad-x8-...,3.195
1,Asus,https://www.vatanbilgisayar.com//asus-vivobook...,16.037
2,Lenovo,https://www.vatanbilgisayar.com//lenovo-tab-m9...,4.000
3,Lenovo,https://www.vatanbilgisayar.com//lenovo-ideapa...,19.701
4,Honor,https://www.vatanbilgisayar.com//honor-pad-x9-...,6.368
5,Samsung,https://www.vatanbilgisayar.com//samsung-galax...,4.972
6,iPad,https://www.vatanbilgisayar.com//9-nesil-yeni-...,10.799
7,Lenovo,https://www.vatanbilgisayar.com//lenovo-ideapa...,18.664
8,Asus,https://www.vatanbilgisayar.com//asus-vivobook...,28.996
9,Samsung,https://www.vatanbilgisayar.com//samsung-galax...,4.972


In [13]:
df.to_csv("./computer_data_new.csv",index=False)